In [4]:
%pip install ipynb

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from ipynb.fs.full.twscrape import *
import pandas as pd

In [2]:
user_name = input("Input the username: ")
limit = int(input("Input the amount of tweets to get: "))
result = await gettweets(user_name, limit)
##print(result)

2023-11-02 13:10:38.106 | WARNING  | twscrape.accounts_pool:add_account:76 - Account BingBong2397074 already exists
2023-11-02 13:10:38.132 | WARNING  | twscrape.accounts_pool:add_account:76 - Account BBong20237297 already exists
2023-11-02 13:10:38.296 | INFO     | twscrape.accounts_pool:login_all:155 - [1/1] Logging in BBong20237297 - twitterscrapertest@rambler.ua
2023-11-02 13:10:39.839 | INFO     | twscrape.accounts_pool:login:138 - Logged in to BBong20237297 successfully


user login: User(id=813286, id_str='813286', url='https://twitter.com/BarackObama', username='BarackObama', displayname='Barack Obama', rawDescription='Dad, husband, President, citizen.', created=datetime.datetime(2007, 3, 5, 22, 8, 25, tzinfo=datetime.timezone.utc), followersCount=132007604, friendsCount=549287, statusesCount=16941, favouritesCount=8, listedCount=215234, mediaCount=2587, location='Washington, DC', profileImageUrl='https://pbs.twimg.com/profile_images/1329647526807543809/2SGvnHYV_normal.jpg', profileBannerUrl='https://pbs.twimg.com/profile_banners/813286/1633800288', protected=None, verified=False, blue=True, blueType=None, descriptionLinks=[TextLink(url='http://obamabook.com', text='obamabook.com', tcourl='https://t.co/kHvnxozw8x')], _type='snscrape.modules.twitter.User') 

user id: 813286 

[Tweet(id=1719072368524476556, id_str='1719072368524476556', url='https://twitter.com/BarackObama/status/1719072368524476556', date=datetime.datetime(2023, 10, 30, 19, 22, 46, tzi

In [5]:
##type(result)

In [3]:
df_tweets = pd.DataFrame(result)

In [6]:
##df_tweets.head()

In [7]:
##for column in df_tweets.columns:
##    print(column, " ", type(column))

In [4]:
## avoid sql keywords
df_tweets.rename(columns={'url': 'tweet_url', 'date': 'publish_date','user': 'tweet_user'}, inplace=True)

In [9]:
##df_tweets.head()

In [6]:
%pip install psycopg2

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 443.3 kB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for psycopg2: filename=psycopg2-2.9.9-cp311-cp311-linux_x86_64.whl size=490641 sha256=a869bde106c9a165068760604aee6e63dad7ea2bb9a16a8cbd7765e35acca3d7
  Stored in directory: /home/vscode/.cache/pip/wheels/ab/34/b9/78ebef1b3220b4840ee482461e738566c3c9165d2b5c914f51
Successfully built psycopg2

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import psycopg2

In [6]:
conn = psycopg2.connect(dbname="postgres",user="postgres", password="postgres", port="5432", host="localhost")
conn.autocommit = True
cursor = conn.cursor()

In [7]:
import sys
import traceback

In [19]:
for index, row in df_tweets.iterrows():
    try:
        cursor.execute('''INSERT INTO Tweets (
                            id,
                            id_str,
                            tweet_url,
                            publish_date,
                            tweet_user,
                            lang,
                            rawcontent,
                            replycount,
                            retweetcount,
                            likecount,
                            quotecount,
                            conversationid,
                            hashtags,
                            cashtags,
                            mentionedusers,
                            links,
                            viewcount,
                            retweetedtweet,
                            quotedtweet,
                            place,
                            coordinates,
                            inreplytotweetid,
                            inreplytouser,
                            source,
                            sourceurl,
                            sourcelabel,
                            media,
                            _type
                        )
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);''', 
                       (row.id,
                        row.id_str,
                        row.tweet_url,
                        row.publish_date,
                        str(row.tweet_user),
                        row.lang,
                        row.rawContent,
                        row.replyCount,
                        row.retweetCount,
                        row.likeCount,
                        row.quoteCount,
                        row.conversationId,
                        str(row.hashtags),
                        str(row.cashtags),
                        str(row.mentionedUsers),
                        str(row.links),
                        row.viewCount,
                        str(row.retweetedTweet),
                        str(row.quotedTweet),
                        str(row.place),
                        str(row.coordinates),
                        str(row.inReplyToTweetId),
                        str(row.inReplyToUser),
                        str(row.source),
                        str(row.sourceUrl),
                        str(row.sourceLabel),
                        str(row.media),
                        str(row._type)
                       )
        )
        
    except psycopg2.errors.UniqueViolation:
        print('Tweet already in database. Tweet ID: ', row.id)
    except BaseException as ex:
            ex_type, ex_value, ex_traceback = sys.exc_info()
            
            # Extract unformatter stack traces as tuples
            trace_back = traceback.extract_tb(ex_traceback)

            # Format stacktrace
            stack_trace = list()

            for trace in trace_back:
                stack_trace.append("File : %s , Line : %d, Func.Name : %s, Message : %s" % (trace[0], trace[1], trace[2], trace[3]))
            print('Error at: ', row.id, 'Error: ', ex_value)
            print("Exception type : %s " % ex_type.__name__)
            print("Exception message : %s" %ex_value)
            print("Stack trace : %s" %stack_trace)
            print('______________')

## ToDo extract only username from tweet_user!!!!!

Tweet already in database. Tweet ID:  1687511710339407874
Tweet already in database. Tweet ID:  1687512224770793480
Tweet already in database. Tweet ID:  1661153116882870272
Tweet already in database. Tweet ID:  1638956749804675109
Tweet already in database. Tweet ID:  1638956750886805506
Tweet already in database. Tweet ID:  1587799447915954176
Tweet already in database. Tweet ID:  1587799460087795723
Tweet already in database. Tweet ID:  1573318948002619393
Tweet already in database. Tweet ID:  1573318953606209536
Tweet already in database. Tweet ID:  1555221890372861952
Tweet already in database. Tweet ID:  1555221895661879297
Tweet already in database. Tweet ID:  1545388676242456576
Tweet already in database. Tweet ID:  1545388677307809793
Tweet already in database. Tweet ID:  1527749317402013696
Tweet already in database. Tweet ID:  1511358084106629120
Tweet already in database. Tweet ID:  1511358091769655296
Tweet already in database. Tweet ID:  1504178334476943362
Tweet already 